In [19]:
# Memanggil library yang dibutuhkan
import ee
import geemap

# Autentikasi dan Inisialisasi
ee.Authenticate()
ee.Initialize()

# Membuat peta interaktif
Map = geemap.Map()

# ======================================================================
# MEMUAT DATA INPUT & MENDEFINISIKAN AOI
# ======================================================================
path_aset_lc_2016 = "users/ivan_taslim/LC_Paguyaman_2016_DW_Filled"
lc_2016 = ee.Image(path_aset_lc_2016)
aoi = lc_2016.geometry()
Map.addLayer(aoi, {'color': 'red'}, 'Batas DAS Paguyaman (AOI)')
Map.centerObject(aoi, 11)

# ======================================================================
# MENGHITUNG FAKTOR RUSLE (DENGAN SUMBER K-FACTOR YANG VALID)
# ======================================================================

# Faktor LS (Lereng)
dem = ee.Image('USGS/SRTMGL1_003')
LS_factor = ee.Terrain.slope(dem).clip(aoi)

# Faktor K (Tanah) - MENGGUNAKAN ASET PUBLIK YANG SUDAH TERVERIFIKASI
K_factor = ee.Image("users/hkv/SoilGrids_K_Factor").clip(aoi)

# Faktor R (Hujan)
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
precipitation_2016 = chirps.filter(ee.Filter.date('2016-01-01', '2016-12-31')).sum().clip(aoi)
R_factor = precipitation_2016.multiply(0.35).add(38.5)

# Faktor C (Tanaman)
reclass_dict = {
    'from': [1, 2, 3, 4, 6],
    'to':   [0.01, 0.6, 0.05, 0.25, 0.02]
}
C_factor_2016 = lc_2016.remap(reclass_dict['from'], reclass_dict['to'])

print("Langkah 1 Selesai: Semua faktor RUSLE telah dihitung ulang dengan sumber K-Factor yang valid.")
Map

Langkah 1 Selesai: Semua faktor RUSLE telah dihitung ulang dengan sumber K-Factor yang valid.


Map(center=[0.7139019338201237, 122.37991662440572], controls=(WidgetControl(options=['position', 'transparent…

In [22]:
# ======================================================================
# PEMERIKSAAN LANJUTAN (JALANKAN SETELAH LANGKAH 1 YANG DIPERBAIKI)
# ======================================================================

# Fungsi untuk mencetak nilai min dan max dari sebuah gambar
def print_min_max(image, name, aoi):
    # Dapatkan nama band pertama untuk pelaporan yang lebih jelas
    band_name = image.bandNames().get(0).getInfo()
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=aoi,
        scale=90, # Skala audit
        maxPixels=1e9
    ).getInfo()
    print(f"Pemeriksaan Faktor {name}: {stats}")

# Lakukan pengecekan untuk setiap faktor yang baru dibuat
print("======================================================================")
print("MEMERIKSA NILAI MINIMUM & MAKSIMUM SETIAP FAKTOR (VERSI BARU)")
print("----------------------------------------------------------------------")
print_min_max(R_factor, 'R (Hujan)', aoi)
print_min_max(K_factor, 'K (Tanah)', aoi)
print_min_max(LS_factor, 'LS (Lereng)', aoi)
print("======================================================================")

MEMERIKSA NILAI MINIMUM & MAKSIMUM SETIAP FAKTOR (VERSI BARU)
----------------------------------------------------------------------
Pemeriksaan Faktor R (Hujan): {'precipitation_max': 735.9519455403089, 'precipitation_min': 438.3214212732389}


EEException: Image.load: Image asset 'users/hkv/SoilGrids_K_Factor' not found (does not exist or caller does not have access).

In [9]:
# Membuat Faktor C baru yang nilainya 50% lebih tinggi
C_factor_sensitivity = C_factor_2016.multiply(1.5)

# Menghitung ulang erosi dengan Faktor C yang baru
erosion_sensitivity = R_factor.multiply(K_factor).multiply(LS_factor).multiply(C_factor_sensitivity).select([0]).rename('erosion')

# Menghitung statistik rata-rata untuk uji sensitivitas
stats_sensitivity = erosion_sensitivity.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=90,
    maxPixels=1e12
)

# Menampilkan hasil uji sensitivitas
avg_erosion_sensitivity = stats_sensitivity.get('erosion').getInfo()
print("======================================================================")
print("UJI SENSITIVITAS (FAKTOR C NAIK 50%)")
print("----------------------------------------------------------------------")
print(f"Nilai Erosi Rata-rata (Uji Sensitivitas): {avg_erosion_sensitivity:.4f} ton/ha/tahun")
print("======================================================================")

UJI SENSITIVITAS (FAKTOR C NAIK 50%)
----------------------------------------------------------------------
Nilai Erosi Rata-rata (Uji Sensitivitas): 0.0000 ton/ha/tahun


In [10]:
# Menghitung histogram dari nilai piksel di peta erosi
histogram_info = soil_erosion_2016.reduceRegion(
    reducer=ee.Reducer.histogram(maxBuckets=30, minValue=0, maxValue=0.1), # Fokus pada nilai sangat kecil
    geometry=aoi,
    scale=90,
    maxPixels=1e12
).get('erosion').getInfo()

# Memproses data untuk plotting
if histogram_info and 'bucketMeans' in histogram_info and 'histogram' in histogram_info:
    x = histogram_info['bucketMeans']
    y = histogram_info['histogram']
    
    # Membuat plot
    plt.figure(figsize=(10, 6))
    plt.bar(x, y, width=0.005)
    plt.title('Histogram Sebaran Nilai Erosi Piksel (Tahun 2016)')
    plt.xlabel('Nilai Erosi (ton/ha/tahun)')
    plt.ylabel('Jumlah Piksel')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()
else:
    print("Tidak dapat membuat histogram, kemungkinan semua nilai piksel identik (nol).")

TypeError: Reducer.histogram() got an unexpected keyword argument 'minValue'

In [11]:
# Fungsi untuk mencetak nilai min dan max dari sebuah gambar
def print_min_max(image, name, aoi):
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=aoi,
        scale=90,
        maxPixels=1e9
    ).getInfo()
    print(f"Pemeriksaan Faktor {name}: {stats}")

# Lakukan pengecekan untuk setiap faktor
print("======================================================================")
print("MEMERIKSA NILAI MINIMUM & MAKSIMUM SETIAP FAKTOR")
print("----------------------------------------------------------------------")
print_min_max(R_factor, 'R (Hujan)', aoi)
print_min_max(K_factor, 'K (Tanah)', aoi)
print_min_max(LS_factor, 'LS (Lereng)', aoi)
print("======================================================================")

MEMERIKSA NILAI MINIMUM & MAKSIMUM SETIAP FAKTOR
----------------------------------------------------------------------
Pemeriksaan Faktor R (Hujan): {'precipitation_max': 735.9519455403089, 'precipitation_min': 438.3214212732389}
Pemeriksaan Faktor K (Tanah): {'silt_0-5cm_mean_max': 0, 'silt_0-5cm_mean_min': 0}
Pemeriksaan Faktor LS (Lereng): {'slope_max': 58.33685888552522, 'slope_min': 0}


In [25]:
# ======================================================================
# AUDIT & PERHITUNGAN EROSI FINAL (DENGAN UNIT YANG BENAR)
# ======================================================================

# Memanggil library yang dibutuhkan
import ee
import geemap

# Autentikasi dan Inisialisasi
ee.Authenticate()
ee.Initialize()
Map = geemap.Map()

# ----------------------------------------------------------------------
# 1. PERSIAPAN DATA DASAR
# ----------------------------------------------------------------------
print("Langkah 1: Memuat data dasar...")
path_aset_lc_2016 = "users/ivan_taslim/LC_Paguyaman_2016_DW_Filled"
lc_2016 = ee.Image(path_aset_lc_2016)
aoi = lc_2016.geometry()
Map.addLayer(aoi, {'color': 'red'}, 'Batas DAS Paguyaman (AOI)')
Map.centerObject(aoi, 11)

# ----------------------------------------------------------------------
# 2. MENGHITUNG FAKTOR K DENGAN KONVERSI UNIT YANG BENAR
# ----------------------------------------------------------------------
print("\nLangkah 2: Menghitung Faktor K dengan unit yang benar...")

# Memuat data tanah primer
sand_raw = ee.Image("projects/soilgrids-isric/sand_mean").select(0).clip(aoi)
silt_raw = ee.Image("projects/soilgrids-isric/silt_mean").select(0).clip(aoi)
clay_raw = ee.Image("projects/soilgrids-isric/clay_mean").select(0).clip(aoi)
org_carbon_raw = ee.Image("projects/soilgrids-isric/soc_mean").select(0).clip(aoi)

# == PERBAIKAN UTAMA: Konversi dari g/kg dan dg/kg ke fraksi desimal ==
sand = sand_raw.divide(1000.0)
silt = silt_raw.divide(1000.0)
clay = clay_raw.divide(1000.0)
org_matter = org_carbon_raw.divide(1000.0) # Organic Matter (OM) = SOC / 10 * 1.724 -> disederhanakan

# Menghitung Faktor K menggunakan rumus Wischmeier & Smith (1978)
A = (silt.multiply(ee.Image(1.0).subtract(clay))).multiply(-0.0256).exp()
B = silt.divide(clay.add(silt))
C = ee.Image(1.0).subtract(ee.Image(0.25).multiply(org_matter).divide(org_matter.add(ee.Image(0.025).multiply(org_matter).multiply(-1).add(3.72)).exp()))
D = ee.Image(1.0).subtract(ee.Image(0.7).multiply(ee.Image(1.0).subtract(sand)).divide(ee.Image(1.0).subtract(sand).add(ee.Image(-5.51).add(ee.Image(22.9).multiply(ee.Image(1.0).subtract(sand)))).exp()))
K_factor = A.multiply(B.pow(1.14)).multiply(C).multiply(D).multiply(0.1317).rename('k_factor')

# ----------------------------------------------------------------------
# 3. MENGHITUNG FAKTOR LAINNYA & EROSI FINAL
# ----------------------------------------------------------------------
print("\nLangkah 3: Menghitung faktor lainnya dan erosi final...")

# Faktor LS
dem = ee.Image('USGS/SRTMGL1_003')
LS_factor = ee.Terrain.slope(dem).clip(aoi)

# Faktor R
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
precipitation_2016 = chirps.filter(ee.Filter.date('2016-01-01', '2016-12-31')).sum().clip(aoi)
R_factor = precipitation_2016.multiply(0.35).add(38.5)

# Faktor C
reclass_dict = {'from': [1, 2, 3, 4, 6], 'to': [0.01, 0.6, 0.05, 0.25, 0.02]}
C_factor_2016 = lc_2016.remap(reclass_dict['from'], reclass_dict['to'])

# Menghitung erosi final
soil_erosion_2016 = R_factor.multiply(K_factor).multiply(LS_factor).multiply(C_factor_2016).rename('erosion')

# ----------------------------------------------------------------------
# 4. MENAMPILKAN HASIL AKHIR
# ----------------------------------------------------------------------
print("\nLangkah 4: Menampilkan hasil akhir...")

stats_final = soil_erosion_2016.reduceRegion(
    reducer=ee.Reducer.mean(), geometry=aoi, scale=90, maxPixels=1e12
)

avg_erosion_final = stats_final.get('erosion').getInfo()
print("======================================================================")
print("HASIL PERHITUNGAN EROSI FINAL (SETELAH PERBAIKAN TOTAL)")
print("----------------------------------------------------------------------")
print(f"Nilai Erosi Rata-rata (presisi tinggi): {avg_erosion_final:.10f} ton/ha/tahun")
print("======================================================================")

# Menampilkan peta erosi di peta interaktif
erosion_palette = ['#32CD32', '#FFFF00', '#FFA500', '#FF4500', '#FF0000']
Map.addLayer(soil_erosion_2016, {'min': 0, 'max': 50, 'palette': erosion_palette}, 'Peta Erosi 2016 (FINAL)')
Map

Langkah 1: Memuat data dasar...

Langkah 2: Menghitung Faktor K dengan unit yang benar...

Langkah 3: Menghitung faktor lainnya dan erosi final...

Langkah 4: Menampilkan hasil akhir...
HASIL PERHITUNGAN EROSI FINAL (SETELAH PERBAIKAN TOTAL)
----------------------------------------------------------------------
Nilai Erosi Rata-rata (presisi tinggi): 26.1697227021 ton/ha/tahun


Map(center=[0.7139019338201237, 122.37991662440572], controls=(WidgetControl(options=['position', 'transparent…

In [26]:
# ======================================================================
# LANGKAH 5: EKSPOR PETA EROSI
# ======================================================================

# --- Pastikan variabel di bawah ini sudah benar sesuai tahun analisis Anda ---
# Ganti dengan variabel hasil erosi yang sesuai (misal: soil_erosion_2016,2020 atau soil_erosion_2024)
gambar_untuk_ekspor = soil_erosion_2016 
# Ganti dengan tahun yang sesuai
TAHUN_EKSPOR = 2016
# -----------------------------------------------------------------------------


# 5.1 Ekspor ke GEE Assets
task_asset = ee.batch.Export.image.toAsset(
  image=gambar_untuk_ekspor.toFloat(),
  description=f'Erosi_Paguyaman_{TAHUN_EKSPOR}_to_Asset',
  assetId=f'users/ivan_taslim/Erosi_Paguyaman_{TAHUN_EKSPOR}',
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_asset.start()
print(f"Proses ekspor Erosi {TAHUN_EKSPOR} ke GEE ASSETS telah dimulai.")

# 5.2 Ekspor ke Google Drive
task_drive = ee.batch.Export.image.toDrive(
  image=gambar_untuk_ekspor.toFloat(),
  description=f'Erosi_Paguyaman_{TAHUN_EKSPOR}_to_Drive',
  folder='Hasil_Analisis_DAS_Paguyaman',
  fileNamePrefix=f'Erosi_Paguyaman_{TAHUN_EKSPOR}',
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_drive.start()
print(f"Proses ekspor Erosi {TAHUN_EKSPOR} ke GOOGLE DRIVE telah dimulai.")
print("\nPERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.")

Proses ekspor Erosi 2016 ke GEE ASSETS telah dimulai.
Proses ekspor Erosi 2016 ke GOOGLE DRIVE telah dimulai.

PERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.
